A scheduling problem. Taken from https://developers.google.com/optimization/assignment/assignment_example


We have 5 five workers and 4 tasks and we want the most cost effective way of assigning the workers to each task. 
The workers, tasks and the associated costs are presented below:

||||||
|--- |--- |--- |--- |--- |
||0|1|2|3|
|0|90|80|75|70|
|1|35|85|55|65|
|2|125|95|90|95|
|3|45|110|95|115|
|4|50|100|90|100|

We are going to use PuLP to find the best assignement


In [2]:
from pulp import *

In [3]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [50, 100, 90, 100],
]
num_workers = len(costs)
num_tasks = len(costs[0])

In [4]:
#Create the problem:
prob = LpProblem("ScheduleProblem", LpMinimize)

# Create a tuple for each worker and each task:
worker_task = [(worker, task) for worker in range(num_workers) for task in range(num_tasks)]

# This time we are using binary variables. Each variable can be seen as a variable worker_task_(x, y) which 
# has a value 1 if worker x is assigned to task y. 
worker_task_vars = LpVariable.dicts("worker_task", worker_task,0, 1, LpInteger)

print(worker_task_vars)

{(0, 0): worker_task_(0,_0), (0, 1): worker_task_(0,_1), (0, 2): worker_task_(0,_2), (0, 3): worker_task_(0,_3), (1, 0): worker_task_(1,_0), (1, 1): worker_task_(1,_1), (1, 2): worker_task_(1,_2), (1, 3): worker_task_(1,_3), (2, 0): worker_task_(2,_0), (2, 1): worker_task_(2,_1), (2, 2): worker_task_(2,_2), (2, 3): worker_task_(2,_3), (3, 0): worker_task_(3,_0), (3, 1): worker_task_(3,_1), (3, 2): worker_task_(3,_2), (3, 3): worker_task_(3,_3), (4, 0): worker_task_(4,_0), (4, 1): worker_task_(4,_1), (4, 2): worker_task_(4,_2), (4, 3): worker_task_(4,_3)}


In [5]:
#As always first add the objective function
prob += lpSum([costs[worker][task] * worker_task_vars[worker, task] for  worker,task in worker_task])

In [6]:
#Each worker can only do one task

for worker in range(num_workers):
    prob += lpSum([worker_task_vars[worker, task] for  task in range(num_tasks)]) <= 1
    
#Each task needs to be done once

for task in range(num_tasks):
    prob += lpSum([worker_task_vars[worker, task] for  worker in range(num_workers)]) == 1


In [42]:
# The problem is solved using PuLP's choice of Solver
prob.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

Status: Optimal
worker_task_(0,_0) = 0.0
worker_task_(0,_1) = 0.0
worker_task_(0,_2) = 0.0
worker_task_(0,_3) = 1.0
worker_task_(1,_0) = 0.0
worker_task_(1,_1) = 0.0
worker_task_(1,_2) = 1.0
worker_task_(1,_3) = 0.0
worker_task_(2,_0) = 0.0
worker_task_(2,_1) = 1.0
worker_task_(2,_2) = 0.0
worker_task_(2,_3) = 0.0
worker_task_(3,_0) = 1.0
worker_task_(3,_1) = 0.0
worker_task_(3,_2) = 0.0
worker_task_(3,_3) = 0.0
worker_task_(4,_0) = 0.0
worker_task_(4,_1) = 0.0
worker_task_(4,_2) = 0.0
worker_task_(4,_3) = 0.0


In [43]:
#Parse result
for worker_task,v in worker_task_vars.items():
    if v.varValue > 0:
        print("worker {} performs task {} ".format(worker_task[0], worker_task[1]))


worker 0 performs task 3 
worker 1 performs task 2 
worker 2 performs task 1 
worker 3 performs task 0 
